In [12]:
import redcap
import pandas as pd
from datetime import datetime, timedelta
data = {
    'token': '085520489DF98911614E03FF05D58EA8',
    'content': 'project',
    'format': 'csv',
    'returnFormat': 'csv'
}

df= redcap.Project('https://ruff.mrc.gm:8443/redcap/api/', data['token'])

data = {
    'token': '085520489DF98911614E03FF05D58EA8',
    'content': 'project',
    'format': 'csv',
    'returnFormat': 'csv'
}

IdeA_Data = redcap.Project('https://ruff.mrc.gm:8443/redcap/api/', data['token'])

In [13]:
df_consent=df.export_records(forms=['consent'])
df_consent=pd.DataFrame(df_consent)

In [14]:
df_consent=df_consent[
    (df_consent['consent_complete']=='2')&
     (df_consent['cons_obtained_q4']=='1')&
   
    (df_consent['dob']=='')

]


In [15]:
type(df_consent['cons_obtained_q4'])

pandas.core.series.Series

In [16]:
df_consent=df_consent[
    (df_consent['cons_obtained_q4']=='1')|
      (df_consent['cons_obtained_q4']=='2')
]

In [17]:
df_screen=df.export_records(forms=['screening'])
df_screen=pd.DataFrame(df_screen)
df_screen=df_screen[
    (df_screen['screening_complete']=='2')
]

In [18]:
df_sysmex=df.export_records(forms=['sysmexresult'])
df_sysmex=pd.DataFrame(df_sysmex)
df_sysmex['sysm_hgb_q3'] = pd.to_numeric(df_sysmex['sysm_hgb_q3'], errors='coerce')

In [19]:
df_sysmex=df_sysmex[
    (df_sysmex['redcap_event_name']=='v1_arm_1')&
    (df_sysmex['sysm_hgb_q3']>11.0)
]


In [20]:
completed_fellow_up=df.export_report(report_id='410')
completed_fellow_up=pd.DataFrame(completed_fellow_up)


# Assuming 'bld_datecoll_q1' is in datetime format, if not, convert it first.
completed_fellow_up['bld_datecoll_q1'] = pd.to_datetime(completed_fellow_up['bld_datecoll_q1'])

# Get the current date
current_date = datetime.now()

# Calculate the difference between 'start_date' and the current date
completed_fellow_up['date_difference'] = current_date - completed_fellow_up['bld_datecoll_q1']
completed_fellow_up.to_csv('completed_fellow_up.csv',index=False)
# Filter out records where the date difference is greater than 14 days
filtered_completed_fellow_up = completed_fellow_up[completed_fellow_up['date_difference'] > timedelta(days=14)]

# Optionally, you can drop the 'date_difference' column if you don't need it
filtered_completed_fellow_up.to_csv('filtered_completed_fellow_up.csv',index=False)

In [ ]:
iron_sup=IdeA_Data.export_records(forms=['ironsupplement'])
iron_sup=pd.DataFrame(iron_sup)

iron_sup=iron_sup[['cons_recordid_q1','redcap_event_name', 'redcap_repeat_instance','iron_visitstatus_q1a','iron_visitreason_q1b']]
iron_sup=iron_sup[
    (iron_sup['iron_visitstatus_q1a']=='0')
]
iron_sup['occurrences'] = iron_sup['cons_recordid_q1'].map(iron_sup['cons_recordid_q1'].value_counts())
iron_sup = iron_sup.drop_duplicates(subset=['cons_recordid_q1'])

In [ ]:
deviation = pd.merge(filtered_completed_fellow_up, iron_sup, on='cons_recordid_q1',how='inner')
deviation.to_csv('completed_deviation.csv',index=False)

bloodsample_processing=IdeA_Data.export_records(forms=['bloodsample_processing'])
bloodsample_processing=pd.DataFrame(bloodsample_processing)

aliqoute1=bloodsample_processing[
    (bloodsample_processing['samp_serumaliquot_q3___1']=='1')
]
aliqoute2=bloodsample_processing[
    (bloodsample_processing['samp_serumaliquot_q3___2']=='1')
]

aliqoute3=bloodsample_processing[
    (bloodsample_processing['samp_serumaliquot_q3___3']=='1')
]
bloodsample_processing=bloodsample_processing[
    (bloodsample_processing['bloodsample_processing_complete']=='2')
]
stool_processing=IdeA_Data.export_records(forms=['stool_processing'])
stool_processing=pd.DataFrame(stool_processing)
stool_processing=stool_processing[
    (stool_processing['stool_processing_complete']=='2')
]
stool_processing_aliquate_1=stool_processing[
    (stool_processing['stoo_aliquot_q6']=='1')
]